In [1]:
import pandas as pd
import numpy as np

In [2]:
city = "paris"
listing = pd.read_csv("data_france/"+city+"/listings.csv")
calendar = pd.read_csv("data_france/"+city+"/calendar.csv")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
calendar["id"] = calendar["listing_id"]

In [4]:
cols_listing = ['id',
                 'zipcode',
                 'property_type',
                 'room_type',
                 'accommodates',
                 'bedrooms',
                 'square_feet',
                 'price',
                 'number_of_reviews',
                 'review_scores_rating']
cols_calendar = ["id", "date", "price", "adjusted_price"]

In [5]:
listing = listing[cols_listing]
calendar = calendar[cols_calendar]

In [6]:
listing[:3]

,id,zipcode,property_type,room_type,accommodates,bedrooms,square_feet,price,number_of_reviews,review_scores_rating
0,33065009,75002,Apartment,Entire home/apt,8,3.0,NaN,$256.00,29,94.0
1,33008241,75003,Apartment,Entire home/apt,3,1.0,NaN,$85.00,15,77.0
2,39024702,75002,Apartment,Entire home/apt,4,1.0,NaN,$88.00,0,NaN


In [7]:
calendar[:3]

,id,date,price,adjusted_price
0,70586,2019-11-08,$139.00,$139.00
1,3109,2019-11-09,$60.00,$60.00
2,3109,2019-11-10,$60.00,$60.00


In [8]:
result = pd.merge(listing, calendar, on='id', how='outer')
result = result[result["price_x"] == result["price_y"]]
result = result[result["number_of_reviews"]>0]
result = result[result["square_feet"]>0]

In [9]:
data = result.dropna().sample(frac=1)

In [10]:
data.shape

(190995, 13)

In [11]:
data[:3]

,id,zipcode,property_type,room_type,accommodates,bedrooms,square_feet,price_x,number_of_reviews,review_scores_rating,date,price_y,adjusted_price
14300958,265648,75004,Apartment,Entire home/apt,2,1.0,250.0,$65.00,156,84.0,2020-09-18,$65.00,$65.00
14323738,2191803,75020,Condominium,Entire home/apt,2,0.0,269.0,$62.00,49,96.0,2020-02-16,$62.00,$62.00
7173221,1078415,75018,Apartment,Entire home/apt,5,2.0,807.0,$115.00,6,96.0,2020-02-11,$115.00,$115.00


In [12]:
holidays_list = ['2019-01-01',
                 '2019-04-21',
                 '2019-04-22',
                 '2019-05-01',
                 '2019-05-08',
                 '2019-05-30',
                 '2019-06-09',
                 '2019-06-10',
                 '2019-07-14',
                 '2019-08-15',
                 '2019-11-01',
                 '2019-11-11',
                 '2019-12-24',
                 '2019-12-25',
                 '2019-12-31',
                 '2020-01-01',
                 '2020-04-12',
                 '2020-04-13',
                 '2020-05-01',
                 '2020-05-08',
                 '2020-05-21',
                 '2020-05-31',
                 '2020-06-01',
                 '2020-07-14',
                 '2020-08-15',
                 '2020-11-01',
                 '2020-11-11',
                 '2020-12-24',
                 '2020-12-25',
                 '2020-12-31']

In [13]:
def is_holiday(date):
    if date in holidays_list:
        return 1
    else:
        return 0

In [14]:
def dollar_to_euro(p):
    return p*0.92

In [15]:
zip_price_m2 = pd.read_csv("data_france/price_m2_zip.csv")
def get_value(surface, zipcode):
    r = zip_price_m2[zip_price_m2["zipcode"]==zipcode]
    if r.shape[0] > 0:
        price_m2 = r.iloc[0]['price_m2']
    else:
        r = zip_price_m2[zip_price_m2["zipcode"]//1000==zipcode//1000]
        price_m2 = r.iloc[0]['price_m2']
    return surface * price_m2

In [19]:
values = []
holidays = []
prices = []
months = []

for index, row in data.iterrows():
    surface = int(row["square_feet"])
    zipcode = int(row["zipcode"])
    date = row["date"]
    price = float(row["adjusted_price"].strip("$").replace(',', ''))
    months = int(date.split("-")[1])
    
    values.append(get_value(surface, zipcode))
    holidays.append(is_holiday(date))
    prices.append(dollar_to_euro(price))
    
data["property_value"] = np.array(values)
data["is_holiday"] = np.array(holidays)
data["price"] = np.array(prices)
data["month"] = np.array(months)

In [20]:
cols = ["month", "is_holiday", "property_type", "room_type", "accommodates", "bedrooms", "square_feet", "property_value", "price"]
final_data = data[cols]
final_data

,month,is_holiday,property_type,room_type,accommodates,bedrooms,square_feet,property_value,price
14300958,3,0,Apartment,Entire home/apt,2,1.0,250.0,11500.0,59.80
14323738,3,0,Condominium,Entire home/apt,2,0.0,269.0,8877.0,57.04
7173221,3,0,Apartment,Entire home/apt,5,2.0,807.0,27438.0,105.80
20477800,3,0,Condominium,Entire home/apt,2,1.0,646.0,21964.0,87.40
9082525,3,0,Apartment,Entire home/apt,2,1.0,409.0,13088.0,92.00
11637037,3,0,Apartment,Entire home/apt,6,3.0,1615.0,69445.0,368.00
23087069,3,0,Apartment,Entire home/apt,2,1.0,366.0,12078.0,66.24
10660368,3,0,Apartment,Entire home/apt,2,1.0,484.0,20328.0,138.00
662335,3,0,Apartment,Entire home/apt,6,3.0,1130.0,37290.0,165.60
1510180,3,0,Apartment,Entire home/apt,4,2.0,872.0,32264.0,110.40


In [ ]:
final_data.shape

In [23]:
np.unique(data["property_type"])

array(['Apartment', 'Bed and breakfast', 'Condominium', 'Guest suite',
       'House', 'Loft', 'Other', 'Serviced apartment', 'Townhouse'],
      dtype=object)

In [24]:
d = pd.read_csv("data.csv")

In [25]:
d

,month,is_holiday,property_type,room_type,accommodates,bedrooms,square_feet,property_value,price
0,7,0,Apartment,Private room,2,1.0,161.0,6118.0,92.00
1,12,0,Apartment,Entire home/apt,2,1.0,431.0,19826.0,124.20
2,3,0,Apartment,Private room,2,1.0,108.0,1188.0,30.36
3,1,0,Apartment,Entire home/apt,2,1.0,151.0,6191.0,62.56
4,11,0,Apartment,Entire home/apt,2,1.0,431.0,16809.0,93.84
5,9,0,Apartment,Entire home/apt,2,1.0,409.0,17587.0,165.60
6,5,0,Apartment,Entire home/apt,3,1.0,431.0,16378.0,50.60
7,7,0,Apartment,Entire home/apt,14,3.0,700.0,24500.0,91.08
8,2,0,Apartment,Entire home/apt,4,2.0,538.0,18830.0,133.40
9,9,0,Apartment,Entire home/apt,4,1.0,614.0,21490.0,101.20
